# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [128]:
problem = '''A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared. 
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. 
'''

## 2. Generate the mathematical model

In [129]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [130]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Production Planning

### Sets and Indices:

*   Products: $i \in I = \{COLI, PEPSA\}$

### Parameters:

*   $a_i$: Units of product $i$ obtained per hectolitre of syrup. ($a_{COLI} = 40$, $a_{PEPSA} = 20$)
*   $p_i$: Selling price per unit of product $i$. (Implicitly defined in the revenue function)
*   $c$: Cost of processing one hectolitre of syrup = 150 dollars
*   $C$: Production line capacity per hour = 7100 units
*   $H$: Hours worked per day = 8
*   $D$: Days worked per week = 5

### Decision Variables:

*   $X_i$: Units of product $i$ produced daily.
*   $Y$: Hectolitres of syrup processed daily.

### Objective Function (Maximize Daily Profit):

```
Maximize Z = (49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2) - 150 * Y
```

### Constraints:

1.  **Syrup Usage**: The amount of syrup used must be sufficient to produce the planned quantities of each product.
    ```
    a_{COLI} * X1 + a_{PEPSA} * X2 <= Y
    ```
2.  **Production Capacity**: The total production cannot exceed the line's capacity.
    ```
    X1 + X2 <= C * H
    ```
3.  **Minimum PEPSA Production**: At least half the amount of PEPSA than COLI must be produced.
    ```
    X2 >= 0.5 * X1
    ```
4.  **Non-negativity**: Production quantities cannot be negative.
    ```
    X1, X2, Y >= 0 
    ```

### Model Summary:

This linear programming model aims to maximize the daily profit of the firm by optimizing the production quantities of COLI and PEPSA, considering the limited syrup, production capacity, and minimum PEPSA production requirements. 


## 3. Generate the pyomo code

In [131]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [132]:
#Showing the code in a formatted way
Markdown(response2.text)

## Pyomo Implementation of Production Planning Model

```python
from pyomo.environ import *

# Create a Concrete Model
model = ConcreteModel()

# Sets and Indices
model.I = Set(initialize=['COLI', 'PEPSA'])

# Parameters 
model.a = Param(model.I, initialize={'COLI': 40, 'PEPSA': 20})  # Sample data provided
model.c = Param(initialize=150)  # Sample data provided
model.C = Param(initialize=7100)  # Sample data provided
model.H = Param(initialize=8)  # Sample data provided
model.D = Param(initialize=5)  # Sample data provided

# Decision Variables
model.X = Var(model.I, domain=NonNegativeReals)
model.Y = Var(domain=NonNegativeReals)

# Objective Function
def profit_rule(model):
    return (49000 * model.X['COLI'] - model.X['COLI']**2 + 30 * model.X['PEPSA'] - 2 * model.X['PEPSA']**2) - 150 * model.Y
model.profit = Objective(rule=profit_rule, sense=maximize)

# Constraints
def syrup_usage_rule(model, i):
    return model.a[i] * model.X[i] <= model.Y
model.syrup_usage = Constraint(model.I, rule=syrup_usage_rule)

def capacity_rule(model):
    return sum(model.X[i] for i in model.I) <= model.C * model.H
model.capacity = Constraint(rule=capacity_rule)

def pepsa_minimum_rule(model):
    return model.X['PEPSA'] >= 0.5 * model.X['COLI']
model.pepsa_minimum = Constraint(rule=pepsa_minimum_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal Production Quantities:")
for i in model.I:
    print(f"\t{i}: {model.X[i]():.2f}")
print(f"Syrup Used: {model.Y():.2f}")
print(f"Maximum Profit: {model.profit():.2f}")
```

**Explanation:**

1.  **Model Creation:** A ConcreteModel is created to represent the problem.
2.  **Sets and Indices:** A set `I` is defined to hold the products (COLI and PEPSA).
3.  **Parameters:** The provided sample data for processing cost, capacity, work hours, and product yields are defined as parameters. 
4.  **Decision Variables:**  `X` represents the units of each product produced daily, and `Y` represents the daily syrup usage. Both are non-negative variables.
5.  **Objective Function:** The `profit_rule` function defines the objective to maximize profit based on the given revenue and cost structure.
6.  **Constraints:** 
    *   `syrup_usage_rule` ensures enough syrup is used to meet production.
    *   `capacity_rule` limits production within the line's capacity.
    *   `pepsa_minimum_rule` enforces the minimum PEPSA production requirement.
7.  **Solve and Print:** The model is solved using the 'glpk' solver, and the optimal production quantities, syrup usage, and maximum profit are printed. 


## 4. Input problem data and try running the generated code

In [134]:
from pyomo.environ import *

# Create a Concrete Model
model = ConcreteModel()

# Sets and Indices
model.I = Set(initialize=['COLI', 'PEPSA'])

# Parameters 
model.a = Param(model.I, initialize={'COLI': 40, 'PEPSA': 20})  # Sample data provided
model.c = Param(initialize=150)  # Sample data provided
model.C = Param(initialize=7100)  # Sample data provided
model.H = Param(initialize=8)  # Sample data provided
model.D = Param(initialize=5)  # Sample data provided

# Decision Variables
model.X = Var(model.I, domain=NonNegativeReals)
model.Y = Var(domain=NonNegativeReals)

# Objective Function
def profit_rule(model):
    return (49000 * model.X['COLI'] - model.X['COLI']**2 + 30 * model.X['PEPSA'] - 2 * model.X['PEPSA']**2) - 150 * model.Y
model.profit = Objective(rule=profit_rule, sense=maximize)

# Constraints
def syrup_usage_rule(model, i):
    return model.a[i] * model.X[i] <= model.Y
model.syrup_usage = Constraint(model.I, rule=syrup_usage_rule)

def capacity_rule(model):
    return sum(model.X[i] for i in model.I) <= model.C * model.H
model.capacity = Constraint(rule=capacity_rule)

def pepsa_minimum_rule(model):
    return model.X['PEPSA'] >= 0.5 * model.X['COLI']
model.pepsa_minimum = Constraint(rule=pepsa_minimum_rule)

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Print the results
print("Optimal Production Quantities:")
for i in model.I:
    print(f"\t{i}: {model.X[i]():.2f}")
print(f"Syrup Used: {model.Y():.2f}")
print(f"Maximum Profit: {model.profit():.2f}")

Optimal Production Quantities:
	COLI: 14338.33
	PEPSA: 7169.17
Syrup Used: 573533.33
Maximum Profit: 308381704.17


## 5. Correct the code to verify model viability (optional)

In [136]:
from pyomo.environ import *

# Create a Concrete Model
model = ConcreteModel()

# Sets and Indices
model.I = Set(initialize=['COLI', 'PEPSA'])

# Parameters 
model.a = Param(model.I, initialize={'COLI': 40, 'PEPSA': 20})  # Sample data provided
model.c = Param(initialize=150)  # Sample data provided
model.C = Param(initialize=7100)  # Sample data provided
model.H = Param(initialize=8)  # Sample data provided
model.D = Param(initialize=5)  # Sample data provided

# Decision Variables
model.X = Var(model.I, domain=NonNegativeReals)


# Objective Function
def profit_rule(model):
    return (49000 * model.X['COLI'] - model.X['COLI']**2 + 30 * model.X['PEPSA'] - 2 * model.X['PEPSA']**2) - 150 * (40*model.X['COLI'] + 20 * model.X['PEPSA'])
model.profit = Objective(rule=profit_rule, sense=maximize)

# Constraints

def capacity_rule(model):
    return sum(model.X[i] for i in model.I) <= model.C * model.H
model.capacity = Constraint(rule=capacity_rule)

def pepsa_minimum_rule(model):
    return model.X['PEPSA'] >= 0.5 * model.X['COLI']
model.pepsa_minimum = Constraint(rule=pepsa_minimum_rule)

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Print the results
print("Optimal Production Quantities:")
for i in model.I:
    print(f"\t{i}: {model.X[i]():.2f}")
print(f"Maximum Profit: {model.profit():.2f}")

Optimal Production Quantities:
	COLI: 13838.33
	PEPSA: 6919.17
Maximum Profit: 287249204.17


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [119]:
print(response.text)

## Mathematical Optimization Model for Facility Location

### Sets and Indices:

*   **I:** Set of customers (i ∈ I)
*   **J:** Set of potential facility locations (j ∈ J)

### Parameters:

*   **f<sub>j</sub>:** Fixed annual activation cost for facility at location j
*   **c<sub>ij</sub>:** Transportation cost per unit of demand for serving customer i from facility j
*   **d<sub>i</sub>:** Annual demand of customer i
*   **u<sub>j</sub>:** Maximum annual service volume capacity of facility at location j 

### Decision Variables:

*   **x<sub>ij</sub>:** Fraction of customer i's demand served by facility at location j (0 ≤ x<sub>ij</sub> ≤ 1)
*   **y<sub>j</sub>:** Binary variable indicating whether a facility is opened at location j (y<sub>j</sub> = 1 if open, 0 otherwise)

### Objective Function: 

Minimize the total cost (fixed activation costs + transportation costs)

```
Minimize Z = ∑_{j∈J} f_j * y_j + ∑_{i∈I} ∑_{j∈J} c_ij * d_i * x_ij
```

### Constraints:

1.  **Demand Satisfac

In [120]:
print(response2.text)

```python
# Import Pyomo library
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Sample Data (replace with your actual data)
I = {1, 2, 3}  # Set of customers
J = {1, 2}  # Set of potential facility locations
f = {1: 1000, 2: 1200}  # Fixed annual activation cost for each facility
c = {(1, 1): 5, (1, 2): 7, (2, 1): 6, (2, 2): 4, (3, 1): 8, (3, 2): 3}  # Transportation cost per unit of demand
d = {1: 200, 2: 300, 3: 250}  # Annual demand of each customer
u = {1: 500, 2: 600}  # Maximum annual service volume capacity of each facility

# Define sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Define parameters
model.f = Param(model.J, initialize=f)
model.c = Param(model.I, model.J, initialize=c)
model.d = Param(model.I, initialize=d)
model.u = Param(model.J, initialize=u)

# Define decision variables
model.x = Var(model.I, model.J, bounds=(0, 1))  # Fraction of customer demand served by each facility
model.y = Var(model.J, within=Binary)  #